In [646]:
#from sklearn.neighbors import NearestNeighbors
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pa
import json as js
import networkx as nx
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

In [647]:
file = open('export.json',)
data = js.load(file)
elements = data["elements"]
file.close()

In [648]:
def haversine(point1, point2):
    point1[0], point1[1], point2[0], point2[1] = map(radians, [point1[0], point1[1], point2[0], point2[1]])

    # haversine formula 
    dlon = point2[0] - point1[0] 
    dlat = point2[1] - point1[1] 
    a = sin(dlat/2)**2 + cos(point1[1]) * cos(point2[1]) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [649]:
#regroupe les coordonnées en couple
nodesMap = {}
for e in elements:
    if (e["type"] == "node"):
        if("tags" in e):
            if("name" in e["tags"]):
                if(e["tags"]["name"].startswith("Aire")):
                    print(e["id"],e["tags"]["name"])
                    point = [e["lat"],e["lon"],1,0]
                else:
                    point = [e["lat"],e["lon"],0,0]
            else:
                point = [e["lat"],e["lon"],0,0]

        else:
            point = [e["lat"],e["lon"],0,0]
        # ajout dans le dictionnaire
        nodesMap.update({e["id"] : point})

print(nodesMap[1092718161])


21462872 Aire de Meillac
21463090 Aire de Bordeaux-Cestas
36133148 Aire de Cocumont
101439017 Aire de Lugos-Est
244598847 Aire d'Aquitaine Nord
245469652 Aire de Thouars
245469945 Aire de Thouars
249778863 Aire des Vignes Nord
255725311 Aire de Saugon
257156240 Aire des Palombières
313750636 Aire de la Gare de Péage d'Arveyres
313753580 Aire des Palombières
315104278 Aire des Vignes Sud
315104887 Aire de la Gare de Péage d'Arveyres
318151675 Aire des Terres de Graves
318182389 Aire des Terres de Graves
458755211 Aire de Cocumont
513717312 Aire de Bordeaux-Cestas
726554853 Aire de l'Estalot
736942879 Aire de Lugos-Ouest
778654326 Aire du Bazadais
987039211 Aire du Bazadais
1091814886 Aire de Bédenac Ouest
1876595132 Aire de Saint-Caprais
1893894292 Aire de Cézac
2587088569 Aire de Boisredon
5761204260 Aire de Saugon
[44.2812476, -0.2325377, 0, 0]


In [650]:
# parcourir les types "way"
# noSimpleGraph : {idNode : {
#        nextNodes : [node1,node2,...],
#        prevNodes : [node1,node2,...]
#}}
# 
noSimpleGraph = {}

def updateGraph(idNode,nextNode,prevNode):
    
    if nextNode != None:
        noSimpleGraph[idNode]["nextNodes"].append(nextNode)
    if prevNode != None:
        noSimpleGraph[idNode]["prevNodes"].append(prevNode)


for nodes in elements:
    if (nodes["type"] == "way"):
        for i in range(len(nodes["nodes"])):
            idNode = nodes["nodes"][i] 

            if idNode not in noSimpleGraph:
                noSimpleGraph.update({idNode : {"nextNodes" :[],
                                       "prevNodes" : []}})

            if i == 0:
               updateGraph(idNode,nodes["nodes"][i+1],None)

            elif i == (len(nodes["nodes"])-1):
                updateGraph(idNode,None,nodes["nodes"][i-1])
            else:
                updateGraph(idNode,nodes["nodes"][i+1],nodes["nodes"][i-1])
            
#print(noSimpleGraph)

print(noSimpleGraph[255307664])


{'nextNodes': [3361832774, 1937251760], 'prevNodes': [9331963846]}


In [651]:
simplifiedGraph = {}

for node, data in noSimpleGraph.items():
    if nodesMap.get(node) and nodesMap[node][2] == 1:
        # Le nœud est une zone d'autoroute, ajoutez-le au graphe simplifié.
        simplifiedGraph[node] = {"nextNodes": [], "prevNodes": []}

print(simplifiedGraph)


def simplifier_graphe(graph, deletedNodes):
    aires = [node for node in graph if nodesMap.get(node) and nodesMap[node][2] == 1]
    for node in deletedNodes:
        voisins = graph[node]["nextNodes"] + graph[node]["prevNodes"]
        for voisin in voisins:
            if voisin in deletedNodes:
                for node in graph[voisin]["nextNodes"]:
                    graph[node]["next"].remove(voisin)
                    graph[node]["nextNodes"].remove(node)
                graph[voisin]["prevNodes"].remove(node)

deletedNodes = [node for node in noSimpleGraph if nodesMap.get(node) and nodesMap[node][2] != 1]
print(deletedNodes)

#simplifier_graphe(noSimpleGraph, deletedNodes)

{778654326: {'nextNodes': [], 'prevNodes': []}, 101439017: {'nextNodes': [], 'prevNodes': []}, 249778863: {'nextNodes': [], 'prevNodes': []}, 313750636: {'nextNodes': [], 'prevNodes': []}, 315104887: {'nextNodes': [], 'prevNodes': []}, 315104278: {'nextNodes': [], 'prevNodes': []}, 245469652: {'nextNodes': [], 'prevNodes': []}, 318151675: {'nextNodes': [], 'prevNodes': []}, 255725311: {'nextNodes': [], 'prevNodes': []}, 1893894292: {'nextNodes': [], 'prevNodes': []}, 5761204260: {'nextNodes': [], 'prevNodes': []}, 21463090: {'nextNodes': [], 'prevNodes': []}, 726554853: {'nextNodes': [], 'prevNodes': []}, 987039211: {'nextNodes': [], 'prevNodes': []}, 313753580: {'nextNodes': [], 'prevNodes': []}, 1876595132: {'nextNodes': [], 'prevNodes': []}, 257156240: {'nextNodes': [], 'prevNodes': []}, 736942879: {'nextNodes': [], 'prevNodes': []}, 2587088569: {'nextNodes': [], 'prevNodes': []}, 1091814886: {'nextNodes': [], 'prevNodes': []}, 244598847: {'nextNodes': [], 'prevNodes': []}, 31818238

In [652]:
def updateSimpleGraph(graph,idNode):
    if idNode not in graph:
        graph.update({idNode : {"nextNodes" :[],
                                       "prevNodes" : []}})
    return graph

In [653]:
def depthFirstSearch(graph, node, nodesMap, newGraph, visitedNodes):
    stack = [(node, None)]  # Utilisez None pour la première distance
    lastStationNode = None
    lastDistance = 0.0  # Gardez la dernière distance mise à jour

    while stack:
        node, prevNodeDistance = stack.pop()

        if node not in visitedNodes:
            visitedNodes.add(node)
            if nodesMap[node][2] == 1:
                updateSimpleGraph(newGraph, node)
                if lastStationNode is not None:
                    newGraph[lastStationNode]["nextNodes"].append([node, lastDistance])
                    newGraph[node]["prevNodes"].append([lastStationNode, lastDistance])
                lastStationNode = node
                lastDistance = 0.0  # Réinitialisez la distance à 0.0 pour une nouvelle aire

            if prevNodeDistance is not None:
                lastDistance += prevNodeDistance  # Ajoutez la distance du nœud précédent

            for nextNode in graph.get(node, {}).get("nextNodes", []):
                if nextNode not in visitedNodes:
                    distance = haversine([nodesMap[node][0], nodesMap[node][1]], [nodesMap[nextNode][0], nodesMap[nextNode][1]])
                    stack.append((nextNode, distance))

newGraph = {}

visitedNodes = set()

for node in noSimpleGraph:
    depthFirstSearch(noSimpleGraph, node, nodesMap, newGraph, visitedNodes)

print(newGraph[313753580])

{'nextNodes': [], 'prevNodes': []}


In [654]:

def updateSimpleGraph(graph,idNode):
    if idNode not in graph:
        graph.update({idNode : {"nextNodes" :[],
                                       "prevNodes" : []}})
    return graph


def simplificationGraph():
    for node in nodesMap.items():
        if node[2]==1 and node[3]==0:
            sta


stack = []
simpleGraph = {}
visitedNodes = set()
def nextStation(node, dist, currentAire):
    print(stack)
    print("node",node)
    #noeud dans le graphe
    nodeInGraph = noSimpleGraph[node]


    if node == 255307664:
        print("je passe par laaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
        for voisin in nodeInGraph["nextNodes"]:
            print(nodesMap[voisin])

    # On ajoute le premier noeud du graphe si vide (1 fois)
    if len(simpleGraph) == 0:
        stack.append(node)
        updateSimpleGraph(simpleGraph, currentAire)

    if stack == []:
        return

    # si on a plus de 1 voisin
    if len(nodeInGraph["nextNodes"]) > 1:
        if node not in stack:
            stack.append(node)

        #dist = dist + haversine([nodesMap[node][0], nodesMap[node][1]], [nodesMap[node][0], nodesMap[node][1]])
        for nextNode in nodeInGraph["nextNodes"]:
            #si le voisin n'est pas marqué comme visité
            if nodesMap[nextNode][3] != 1:
                if nodesMap[nextNode][2] == 1:
                    #calculer distance voisins
                    simpleGraph[currentAire]["nextNodes"].append([nextNode,dist])
                    #return dist, nextNode # continuer jusqu'à stack vide
                    nextStation(stack[len(stack)-1], dist, currentAire)
                else :
                    nodesMap[nextNode][3] = 1
                    nextStation(nextNode, dist, currentAire)
        #supprimer de la stack le dernier element
        if stack != []:
            stack.pop()
        if stack == []:
            return nextNode,dist
        nextStation(stack[len(stack)-1], dist, currentAire)

    # Si le noeud n'a qu'un seul voisin ou none
    else:
        # si le voisin est None
        if not noSimpleGraph[node]["nextNodes"]:
            nextStation(stack[len(stack)-1], dist, currentAire)#prendre la derniere de stack

        #stock le voisin dans une variable
        if noSimpleGraph[node]["nextNodes"] != []:
            nextNode = noSimpleGraph[node]["nextNodes"][0]


            #si le voisin est une aire
            if nodesMap[nextNode][2] == 1:
                simpleGraph[currentAire]["nextNodes"].append([nextNode,dist])
                nextStation(stack[len(stack)-1], dist, currentAire)
                #return dist, nextNode

            #si le voisin n'est pas une aire
            #dist = dist + haversine([nodesMap[node][0], nodesMap[node][1]], [nodesMap[nextNode][0], nodesMap[nextNode][1]])
            nextStation(nextNode, dist, currentAire)

    if stack == []:
        print("nodeFinal",node)
        print("stack vide")
        return
dist,nextNode = nextStation(313753580,0,313753580)
visitedNodes.add(nextNode)
print(simpleGraph)
print(nextNode)
print(dist)

[]
node 313753580
[313753580]
node 1670588800
[313753580]
node 1474561851
[313753580]
node 249778271
[313753580]
node 1474561866
[313753580]
node 1474561871
[313753580]
node 249778243
[313753580]
node 1474561925
[313753580]
node 298601381
[313753580]
node 249778298
[313753580]
node 1474562040
[313753580]
node 249778318
[313753580]
node 1474562114
[313753580]
node 11037826493
[313753580]
node 249778328
[313753580]
node 1474562120
[313753580]
node 1474562118
[313753580]
node 249778227
[313753580]
node 1474562099
[313753580]
node 1474562085
[313753580]
node 249778330
[313753580]
node 1474561966
[313753580]
node 1474545625
[313753580]
node 249778321
[313753580]
node 1474545600
[313753580]
node 249778307
[313753580]
node 1474545553
[313753580]
node 249778270
[313753580]
node 1474545539
[313753580]
node 249778338
[313753580]
node 1474545555
[313753580]
node 249778331
[313753580]
node 1474545598
[313753580]
node 1474545607
[313753580]
node 249778320
[313753580]
node 1474540211
[313753580]
nod

21465923
[313753580, 504459143, 332276732]
node 1474545585
[313753580, 504459143, 332276732]
node 21465922
[313753580, 504459143, 332276732]
node 1474561903
[313753580, 504459143, 332276732]
node 1474561961
[313753580, 504459143, 332276732]
node 21465921
[313753580, 504459143, 332276732]
node 1474562083
[313753580, 504459143, 332276732]
node 1474562095
[313753580, 504459143, 332276732]
node 21465920
[313753580, 504459143, 332276732]
node 1474562104
[313753580, 504459143, 332276732]
node 1474562116
[313753580, 504459143, 332276732]
node 21465919
[313753580, 504459143, 332276732]
node 1474562101
[313753580, 504459143, 332276732]
node 21465918
[313753580, 504459143, 332276732]
node 1474562010
[313753580, 504459143, 332276732]
node 21465917
[313753580, 504459143, 332276732]
node 1474561941
[313753580, 504459143, 332276732]
node 1474561919
[313753580, 504459143, 332276732]
node 21465916
[313753580, 504459143, 332276732]
node 332276732
[313753580, 504459143, 332276732]
node 332276734
[313753

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
def stationsOnWay(graph, way, stations):
    visited = set() # Créez un ensemble pour stocker les nœuds visités
    stack = [way[0]] # Créez une pile pour stocker les nœuds à visiter
    print(f"Chemin initialisé au noeud {way[0]}.") # Affichez le nœud de départ

    while stack: # Tant que la pile n'est pas vide
            current_node = stack.pop()  # Nœud en haut de la pile
            visited.add(current_node)  # Marquer le nœud comme visité

            # Vérifier si le nœud courant est dans la liste des stations
            if current_node in stations:
                print(f"La station {current_node} se trouve sur le chemin.")

            # Si le nœud courant est le nœud de fin, recherche terminée
            if current_node == way[1]:
                print(f"Noeud {way[1]} atteint, chemin parcouru.")
                break

            # Ajouter les nœuds voisins non visités à la pile
            for next_node in graph[current_node]['nextNodes']:
                if next_node not in visited:
                    stack.append(next_node)


graph = noSimpleGraph
way = [9576155790, 7509397404]
stations = [node_id for node_id, data in nodesMap.items() if data[2] == 1]